Structure of the analysis is as follows...

- We define up two rank based loss functions: these are functions of the predicted ranks of the 'active' molecules. The first only considers the rank of the highest ranked active, the second considers the average of all the ranks of the actives.
- We do standard 5-fold cross-validation and bootstrap cross-validation. Out of sample predictions are evaluated using the above active-rank loss functions and mean squared error.
- We progressively lower the threshold activity (in terms of the quantile of the activity distribution) and fit the models on a boostrapped datasets with only molecules that have activity beneath this threshold.
- We look at additive loss functions on which to train (penalize) the models better reflecting the out of sample loss.


## Inputs and main functions

In [1]:
from __future__ import division, print_function

import joblib
import numpy as np
from IPython.display import SVG
from scipy.spatial.distance import pdist, squareform, jaccard, cityblock
from scipy import stats

from multiprocessing import Pool
from copy import deepcopy

from sklearn.metrics import mean_squared_error, r2_score
from sklearn import linear_model as LM

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, RidgeCV, BayesianRidge, ElasticNet, Lasso

from keras.models import Sequential
from keras.layers import Dense, Activation

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.svm import SVR

# Deep learning model with intermediate layer...
def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(128, input_dim=128, kernel_initializer='normal', activation='relu'))
    model.add(Dense(16, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=100, batch_size=5, verbose=0)))

# Model dictionaries 
model_dict = {'ridge' : {'m' : Ridge, 'kw' : {'fit_intercept':True, 'alpha':0.1}, },
              'rf'    : {'m' : RandomForestRegressor, 'kw' : {'n_estimators':100, 
                                                              'n_jobs':4, 'max_depth':10}, },
              'svr'   : {'m' : SVR, 'kw' : {}, },
              
              'dl_l'   : {'m' : Pipeline, 
                          'kw' : {'steps': [('standardize', StandardScaler()),
                                            ('mlp', KerasRegressor(build_fn=larger_model, 
                                                                   epochs=100, batch_size=5, 
                                                                   verbose=0))
                                                           ]},
                         },
             }

# Datasets
from glob import glob
targets = [s.replace('datasets/', '') for s in glob('datasets/*')]

import matplotlib.pyplot as plt
%pylab inline
figsize(20, 10)
import seaborn as sns

Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


In [2]:
# Runs without deep learning are much faster - this controls use of DL
do_dl = False

In [3]:
# Arguments for overall number of runs
if not do_dl:
    # Take out when we want to run deep learning model
    del model_dict['dl_l']
    tot_num_runs = 400
    outdir = 'models_final/'
    fig_dir = 'figures/'
    ['svr', 'rf', 'ridge']
    model_labels = ['Support Vector Regression','Random Forests','Ridge Regression']
else:
    tot_num_runs = 100
    outdir = 'models_final_dl/'
    fig_dir = 'figures_dl/'
    model_labels = ['Support Vector Regression','Random Forests','Ridge Regression','Deep Learning']

In [4]:
# Pull in data for a single target name
def get_data(tgt_name='COX-2'):
    data_dir = 'datasets/' + tgt_name + '/'
    preds = joblib.load(data_dir + tgt_name + '_predsu.npy')
    resps = joblib.load(data_dir + tgt_name + '_respu.npy')
    smiles = joblib.load(data_dir + tgt_name + '.smiu')
    return preds, resps, smiles

In [5]:
# Write the loss functions 

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

def avg_mse(predictions, responses, **kwargs):
    return mean_squared_error(responses, predictions) / mean_squared_error(responses, np.zeros_like(responses))


def Rank_loss(predictions, responses, **kwargs):
    
    tgt_val = kwargs.get('tgt_val')
    ranked = np.argsort(-predictions)
    found = responses[ranked] >= tgt_val
    # Number of actives
    N_gamma = np.sum(found)
    
    # Size of test sets
    N_test = predictions.shape[0]
    lt = kwargs.get('loss_type')
    #pdb.set_trace()
    if lt == 'min':
        # Equation (1) of the paper
        loss = 1/(N_test - N_gamma) * np.min(np.arange(N_test)[found])
    elif lt == 'avg':
        # Equation (2) of the paper
        loss = 1/N_gamma * 1/(N_test - N_gamma) * (np.sum(np.arange(N_test)[found]) - N_gamma * (N_gamma - 1)/2)
        pass
    
    assert loss >= 0
    assert loss <= 1
    return loss


def Active_rank_loss_avg(predictions, responses, **kwargs):
    kwargs.update({'loss_type' :'avg'})
    return Rank_loss(predictions, responses, **kwargs)

def Active_rank_loss_min(predictions, responses, **kwargs):
    kwargs.update({'loss_type' :'min'})
    return Rank_loss(predictions, responses, **kwargs)


def nbs_run(kwargs):
    my_is = kwargs.get('is')
    my_oos = kwargs.get('oos')
    method = kwargs.get('method')
    preds = kwargs.get('preds') + 0.
    resps = kwargs.get('resps')
    
    mdl = model_dict[method]['m'](**model_dict[method]['kw'])
    mdl.fit(preds[my_is], resps[my_is])
        
    predictions = mdl.predict(preds[my_oos])
    
    losses = {}
    for (l, v) in loss_dict.iteritems():
        
        # Worth noting, we're looking for the top frac_find _in_the_oos_data_
        # (not in the whole data - as otherwise we might be looking for something
        # that isn't there)
        if 'frac_find' in v['kw']:
            N = len(resps[my_oos])
            sorted_indices = np.argsort(resps[my_oos])
            n = int(N * v['kw']['frac_find'])
            tgt_val = resps[my_oos][sorted_indices[n]]
            v['kw'].update({'tgt_val' : tgt_val})
            
        losses[l] = v['func'](predictions, resps[my_oos], **v['kw'])
        pass
    
    return losses
    
    
def full_bootstrap(preds, resps, method, num_runs=100, insample=False,
                  frac_fit=1.0, num_kf=0, use_pool=True):

    # losses is going to be a list of dicts, loss_type => value
    losses = []
    sorted_indices = np.argsort(resps)

    N = len(resps)
    M = int(N * frac_fit)
    
    # Choose insample values...
    idx_list = []
    for i in range(num_runs):
        if num_kf > 0:
            kf = KFold(n_splits=num_kf, shuffle=True)
            for (tr_i, tst_i) in kf.split(np.arange(M)):
                idcs = sorted_indices[tr_i]
                pass
            pass
        else:
            idcs_rand = np.random.choice(M, M)
            idcs = sorted_indices[idcs_rand]
            pass
        idx_list.append({'is' : idcs})
        pass
    
    # Add in all the other data...
    for d in idx_list:
        if insample:
            oos = d['is']
        else:
            oos = np.delete(np.arange(N), d['is'])
            pass
        d.update({'oos' : oos, 'preds' : preds, 
                  'resps' : resps, 'method' : method})
        pass
            
    if use_pool:
        p = Pool(7)
        losses = p.map(nbs_run, idx_list)
        p.close()
        p.join()
    else:
        losses = [nbs_run(x) for x in idx_list]
        pass
    return losses

In [6]:
# Unit tests for the loss functions...
responses = np.arange(10)
tgt_val = 8

# get it totally right
predictions = np.arange(10)

print('All correct')
print('r2', avg_mse(predictions, responses, tgt_val=tgt_val))
print('Active rank min', Active_rank_loss_min(predictions, responses, tgt_val=tgt_val))
print('Active rank avg', Active_rank_loss_avg(predictions, responses, tgt_val=tgt_val))

# get it totally right where it counts, but maximally wrong elsewhere
#predictions = np.asarray([0])

# one totally right, one totally wrong
predictions = np.roll(np.arange(10),-1)
print('\nOne right, one very wrong')
print('r2', avg_mse(predictions, responses, tgt_val=tgt_val))
print('Active rank min', Active_rank_loss_min(predictions, responses, tgt_val=tgt_val))
print('Active rank avg', Active_rank_loss_avg(predictions, responses, tgt_val=tgt_val))


# get everything wrong
predictions = np.arange(10)[::-1]
print('\nAll wrong')
print('r2', avg_mse(predictions, responses, tgt_val=tgt_val))
print('Active rank min', Active_rank_loss_min(predictions, responses, tgt_val=tgt_val))
print('Active rank avg', Active_rank_loss_avg(predictions, responses, tgt_val=tgt_val))
# 

All correct
r2 0.0
Active rank min 0.0
Active rank avg 0.0

One right, one very wrong
r2 0.315789473684
Active rank min 0.0
Active rank avg 0.5

All wrong
r2 1.15789473684
Active rank min 1.0
Active rank avg 1.0


In [7]:
def summarize(results):
    summary = {}
    for l in loss_dict.keys():
        vals = np.asarray([x[l] for x in results])
        if len(vals) == 1:
            summary[l] = {'loss' : vals[0]}
        else:
            summary[l] = {'loss_l' : np.percentile(vals, 5),
                          'loss' : np.mean(vals),                          
                          'loss_u' : np.percentile(vals, 95),
                         }
            pass
        pass
    return summary

In [8]:
# This function takes the vector of observed values and their mean (implies fewer computations of the mean)
def jackknife(vals, vals_bar):
    n = len(vals)
    var_JK = 0.0
    for v in vals:
        vals_bar_i = (n/(n-1)) * (vals_bar - v/n)
        #print(pow(vals_bar_i - vals_bar, 2))
        var_JK += pow(vals_bar_i - vals_bar, 2)
        pass

    sd_JK = pow( ((n-1)/n) * var_JK, .5)
   
    return sd_JK
    
def jackknife_summary(results):
    summary = {}
    for l in loss_dict.keys():
        vals = np.asarray([x[l] for x in results])
        if len(vals) == 1:
            summary[l] = {'loss' : vals[0]}
        else:
            vals_bar = np.mean(vals)
            sd = jackknife(vals, vals_bar)
            summary[l] = {'loss_l' : vals_bar - 2*sd,
                          'loss' : vals_bar,                          
                          'loss_u' : vals_bar + 2*sd,
                         }
            pass
        pass
    return summary

In [9]:
sorted_targets = sorted(targets, key=lambda x: len(get_data(x)[0]))

In [10]:
import re
gg = 'iuhid.ijgbdbs'
gg = re.sub('[.]','', gg)
print(gg)

iuhidijgbdbs


In [11]:
import os.path
import re

# define the dictionary of losses used here
loss_dict = {'mse' :       {'func' : avg_mse,         'kw' : {}},
             'loss_min_90' :   {'func' : Active_rank_loss_min,  'kw' : {'frac_find' : 0.9}},
             'loss_avg_90' :   {'func' : Active_rank_loss_avg,  'kw' : {'frac_find' : 0.9}},
             'loss_min_95' :   {'func' : Active_rank_loss_min,  'kw' : {'frac_find' : 0.95}},
             'loss_avg_95' :   {'func' : Active_rank_loss_avg,  'kw' : {'frac_find' : 0.95}},
             'loss_min_99' :   {'func' : Active_rank_loss_min,  'kw' : {'frac_find' : 0.99}},
             'loss_avg_99' :   {'func' : Active_rank_loss_avg,  'kw' : {'frac_find' : 0.99}},
                           }
def get_fn(frac_fit, kf=0, insample=False, fig=False, loss=None):
    fn = 'loss_' + str(frac_fit)
    if kf > 0:
        fn += '_kf_' + str(kf)
        pass
    if insample:
        fn += '_insample'
        pass
    od = fig_dir if fig else outdir
    ffn = os.path.join(od, fn)
    if fig:
        ffn = re.sub('[.]','', ffn)
        ffn = ffn + '_' + loss + '.pdf'
        pass
    return ffn
        

def runner(frac_fit=1.0, kf=0, insample=False, use_pool=True):

    fnf = get_fn(frac_fit, kf=kf, insample=insample)
    if os.path.isfile(fnf) and not force_rerun:
        print('Already computed')
        return

    loss_hdr = {}
    if insample:
        num_runs = 1
    elif kf > 0:
        num_runs = int(tot_num_runs/kf)
    else:
        num_runs = tot_num_runs
    
    for tgt in sorted_targets:
        loss_hdr[tgt] = {}
        print ('Doing', tgt)
        preds, resps, _ = get_data(tgt)
        preds = preds + 0.
        for m in model_dict.keys():
            res = full_bootstrap(preds, resps, m, frac_fit=frac_fit, num_kf=kf,
                                 insample=insample, num_runs=num_runs, use_pool=use_pool,
                                )
            joblib.dump(res, outdir + '/detail/' + tgt + '_' + m + '_' + str(frac_fit) + '_' + str(kf) + '.res')
            loss_hdr[tgt][m] = jackknife_summary(res)
        pass
    joblib.dump(loss_hdr, fnf)
    print('Completed')
    return

In [12]:
import matplotlib.pyplot as plt
%pylab inline
figsize(20, 10)
import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


In [13]:
force_rerun = False

In [14]:
# insample run...
runner(insample=True)

Already computed


In [15]:
# Full OOS run...
runner()

Already computed


In [16]:
# OOS run with 5-fold CV for comparison
runner(kf=5)

Already computed


In [17]:
# OOS - max activity at 0.9
runner(frac_fit=0.9)

Already computed


In [18]:
# OOS - max activity at 0.8
runner(frac_fit=0.8)

Already computed


In [19]:
# OOS - max activity at 0.6
runner(frac_fit=0.6)

Already computed


In [20]:
# OOS - max activity at 0.4
runner(frac_fit=0.4)

Already computed


In [21]:
model_dict.keys()

['svr', 'rf', 'ridge']

In [22]:
# Plots results
# Added put_title arg for plots for paper
def plotter(frac_fit=1.0, kf=0, insample=False, loss='mse', save=True, put_title=False, put_grid=False):
    
    fnf = get_fn(frac_fit, kf=kf, insample=insample)
    
    loss_hdr = joblib.load(fnf)
   
    for (i, method) in enumerate(model_dict.keys()):
        losses = np.asarray([loss_hdr[x][method][loss]['loss'] for x in sorted_targets])
        if insample:
            plot(np.arange(25) + 0.05*i, losses,  label=method.upper())
        else:
            loss_l = np.asarray([loss_hdr[x][method][loss]['loss_l'] for x in sorted_targets])
            loss_u = np.asarray([loss_hdr[x][method][loss]['loss_u'] for x in sorted_targets])
            
            yerr = np.vstack((losses - loss_l, loss_u - losses))
            
            errorbar(np.arange(25)+i*0.1, losses, capsize=10, yerr=yerr, label=method.upper())
            pass
        pass
        
    if insample:
        if put_title:
            title('Insample %s Loss' % loss.upper())
    else:
        ttl = 'OOS %s Loss' % loss.upper()
        if kf > 0:
            ttl += ' with %d fold CV' % kf
        if frac_fit < 1.0:
            ttl += ' Max activity in fit at %.1f' % frac_fit
        if put_title:
            title(ttl)
    pass

    grid(put_grid)
    plt.tick_params(top=False, right=False)
    plt.xticks(np.arange(25), sorted_targets, rotation=-45)
    plt.legend(loc='best', fontsize = 'x-large', labels=model_labels)
    if loss=='mse':
        plt.ylabel('Expected mean squared error', fontsize='x-large')
    else:
        plt.ylabel('Expected loss', fontsize='x-large')
    
    if save:
        fnff = get_fn(frac_fit, kf=kf, insample=insample, fig=True, loss=loss)
        print(fnff)
        savefig(fnff, bbox_inches='tight')

### In-sample loss plots

In [ ]:
# We plot the in-sample MSE loss
plotter(insample=True)

figures/loss_10_insample_mse.pdf


In [ ]:
# We plot the in-sample active-rank min loss, with gamma = 0.9
plotter(insample=True, loss='loss_min_90')

In [ ]:
# We plot the in-sample active-rank min loss, with gamma = 0.99
plotter(insample=True, loss='loss_min_99')

In [ ]:
# We plot the in-sample active-rank average loss, with gamma = 0.9
plotter(insample=True, loss='loss_avg_90')

### Out-of-sample (random partitioning) loss plots (using CV)

In [ ]:
# 5-fold CV, scored by MSE
plotter(kf=5)

In [ ]:
# 5-fold CV, scored by active-rank average loss, gamma = 0.9
plotter(kf=5, loss='loss_avg_90')

### Out-of-sample (random partitioning) loss plots (bootstrap)

In [ ]:
# bootstrap CV, scored by MSE
plotter()

In [ ]:
# bootstrap CV, scored by active-rank average loss, gamma = 0.9
plotter(loss='loss_avg_90')

In [ ]:
# bootstrap CV, scored by active-rank min loss, gamma = 0.9
plotter(loss='loss_min_90')

### Out-of-sample loss plots ( training on thresholded data: q=0.9 )

In [ ]:
# Training set activity thresholded at q=0.9, scored by MSE
plotter(frac_fit=0.9)

In [ ]:
# Training set activity thresholded at q=0.9, scored by active-rank average loss, gamma = 0.9
plotter(frac_fit=0.9, loss='loss_avg_90')

In [ ]:
# Training set activity thresholded at q=0.9, scored by active-rank min loss, gamma = 0.9
plotter(frac_fit=0.9, loss='loss_min_90')

### Out-of-sample loss plots ( training on thresholded data: q=0.8 )

In [ ]:
# Training set activity thresholded at q=0.8, scored by MSE
plotter(frac_fit=0.8)

In [ ]:
# Training set activity thresholded at q=0.8, scored by active-rank average loss, gamma = 0.9
plotter(frac_fit=0.8, loss='loss_avg_90')

In [ ]:
# Training set activity thresholded at q=0.8, scored by active-rank min loss, gamma = 0.9
plotter(frac_fit=0.8, loss='loss_min_90')

### Out-of-sample loss plots ( training on thresholded data: q=0.6 )

In [ ]:
# Training set activity thresholded at q=0.6, scored by MSE
plotter(frac_fit=0.6)

In [ ]:
# Training set activity thresholded at q=0.6, scored by active-rank average loss, gamma = 0.9
plotter(frac_fit=0.6, loss='loss_avg_90')

In [ ]:
# Training set activity thresholded at q=0.6, scored by active-rank min loss, gamma = 0.9
plotter(frac_fit=0.6, loss='loss_min_90')

### Out-of-sample loss plots ( training on thresholded data: q=0.4 )

In [ ]:
# Training set activity thresholded at q=0.4, scored by MSE
plotter(frac_fit=0.4)

In [ ]:
# Training set activity thresholded at q=0.4, scored by active-rank average loss, gamma = 0.9
plotter(frac_fit=0.4, loss='loss_avg_99')

In [ ]:
# Training set activity thresholded at q=0.4, scored by active-rank min loss, gamma = 0.9
plotter(frac_fit=0.4, loss='loss_min_90')

In [ ]:
# Training set activity thresholded at q=0.4, scored by active-rank min loss, gamma = 0.99
plotter(frac_fit=0.4, loss='loss_min_99')

## Overall Results

In [ ]:
def probability_min(means, sigmas, K=1000, epsilon = 0.01):
    
    N = len(means)
    probs = np.zeros(N) # array of probability weights
    
    # check that none of the sigmas are zero to avoid NANs
    for ss in range(N):
        if(sigmas[ss]==0):
            sigmas[ss] = epsilon
            pass
        pass

    for i in range(N):
        xs = np.random.normal(loc=means[i],scale=sigmas[i],size=K)
        logterms = np.zeros(K)
        # Iterate over random draw from the i^th normal distribution
        for sim in range(K):
            x = xs[sim]
            logproductterm = 0
            for j in range(N):
                if(j != i):
                    logproductterm += stats.norm.logcdf(-(x-means[j])/sigmas[j]) 
                    pass
                pass
            logterms[sim] = logproductterm
            pass
        probs[i] = np.mean(np.exp(logterms))
        pass
    return probs

# Total score : this is change of ll_estimate
def model_score(frac_fit=1.0, losses=['loss_avg_90', 'mse', ], kf=0, K=1000, recompute_scores=False):
    if recompute_scores:
        fnf = get_fn(frac_fit, kf=0)
        loss_hdr = joblib.load(fnf)

        print(('%9s |' + '%9s |' * len(losses)) % tuple(['',] + losses))
        print('-' * (11 * (len(losses) + 1) - 1))

        M = len(model_dict.keys()) # number of models

        Loss_scores = []

        models = model_dict.keys()
        d = {'losses' : map(lambda x : x + '_' + str(frac_fit), losses)}

        for loss in losses:
            scores = np.zeros(M)
            for tgt in sorted_targets:
                tgt_means = []
                tgt_sigmas = []
                # For each model we extract the mean and SD
                for (i, method) in enumerate(models):
                    lmean = loss_hdr[tgt][method][loss]['loss']
                    llow = loss_hdr[tgt][method][loss]['loss_l']
                    lsigma = (lmean - llow)/2
                    tgt_means.append(lmean)
                    tgt_sigmas.append(lsigma)
                    pass
                # The arrays of means and SDs are used to compute probability of min expected loss
                weights = probability_min(tgt_means, tgt_sigmas, K=K)
                # A weight of 1: min loss with probability 1; weight of 0: min loss with probability 0

                scores += weights   
                pass
            Loss_scores.append(scores)



        for (i, method) in enumerate(models):
            d[method] = np.ravel(map(lambda x: x[i], Loss_scores))
            out = [method,]
            for (j, loss) in enumerate(losses):
                out.append(Loss_scores[j][i])
                pass
            print (('%9s |' + '%9.1f |' * len(losses)) % tuple(out))

        return d
    else:
        print('done this already, skipping...')

In [ ]:
# Change this when the models are rerun
recompute_scores = False

d_oos = model_score(losses=loss_dict.keys(), recompute_scores=recompute_scores)

In [ ]:
import pandas as pd

In [ ]:
d_09 = model_score(losses=loss_dict.keys(), frac_fit=0.9)

In [ ]:
d_08 = model_score(losses=loss_dict.keys(), frac_fit=0.8)

In [ ]:
d_06 = model_score(losses=loss_dict.keys(), frac_fit=0.6)

In [ ]:
d_04 = model_score(losses=loss_dict.keys(), frac_fit=0.4)

In [ ]:
if recompute_scores:
    pd0 = pd.DataFrame(d_oos)
    for d in (d_09, d_08, d_06, d_04):
        pdd = pd.DataFrame(d)
        pd0 = pd0.append(pdd, ignore_index=True)
        pass
    pd0
    if do_dl:
        pd0.to_csv('models_final_dl/loss_summary.csv')
    else:
        pd0.to_csv('models_final/loss_summary.csv')